In [5]:
#lastest code'
import cv2
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# --------------------------
# CONFIG: Give folder containing input images
# --------------------------
input_folder = r"C:\Users\3kp05\Desktop\Neural_Biopsy-main\new semester\all fascicles\set2_Image_01_20x_bf_05_all"
# output_root = r"C:\Users\3kp05\Downloads\removed if 80 inside blur"
# output_root = r"C:\Users\3kp05\Downloads\removed if completely inside blur"
output_root = r"C:\Users\3kp05\Downloads\removed if touches blur"
os.makedirs(output_root, exist_ok=True)

# Scale factor (µm/px)
scale_factor = 0.136

# --------------------------
# Tunables (UPDATED)
# --------------------------
TISSUE_OVERLAP_MIN = 0.50
MAX_MEAN_INTENSITY = 245

MIN_CIRCULARITY = 0.05        # lower to detect small axons
MIN_CONTOUR_AREA = 8          # reduced from 20
MAX_CONTOUR_AREA = 2000000
MIN_SOLIDITY = 0.15           # reduced from 0.3

# --------------------------
# Helpers
# --------------------------
def darken_and_sharpen(image):
    darkened = cv2.convertScaleAbs(image, alpha=1.5, beta=-20)
    sharpen_kernel = np.array([[0, -1, 0],
                               [-1, 5, -1],
                               [0, -1, 0]])
    return cv2.filter2D(darkened, -1, sharpen_kernel)

def build_tissue_mask(full_image):
    hsv = cv2.cvtColor(full_image, cv2.COLOR_BGR2HSV)
    H, S, V = cv2.split(hsv)
    sat_mask = cv2.inRange(S, 15, 255)
    not_too_bright = cv2.inRange(V, 0, 250)

    mask = cv2.bitwise_and(sat_mask, not_too_bright)
    k = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))

    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, k, iterations=1)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, k, iterations=2)
    return mask

def contour_overlap_ratio(contour, mask):
    c_mask = np.zeros(mask.shape, dtype=np.uint8)
    cv2.drawContours(c_mask, [contour], -1, 255, -1)
    inter = cv2.bitwise_and(c_mask, mask)
    area_c = cv2.countNonZero(c_mask)
    area_i = cv2.countNonZero(inter)
    return area_i / float(area_c) if area_c > 0 else 0.0

def mean_intensity_in_contour(gray, contour):
    m = np.zeros(gray.shape, dtype=np.uint8)
    cv2.drawContours(m, [contour], -1, 255, -1)
    return cv2.mean(gray, mask=m)[0]

def circularity(contour):
    a = cv2.contourArea(contour)
    p = cv2.arcLength(contour, True)
    return 4 * np.pi * a / (p * p + 1e-6) if p > 0 else 0

# --------------------------
# Main patch processing
# --------------------------
def process_patch(patch, mask_patch, x_offset, y_offset, axon_data, object_counter, output_image):

    patch = darken_and_sharpen(patch)
    gray = cv2.cvtColor(patch, cv2.COLOR_BGR2GRAY)

    # Small smoothing for better boundary separation
    smooth = cv2.bilateralFilter(gray, 5, 20, 20)

    # CLAHE
    clahe = cv2.createCLAHE(clipLimit=2.5, tileGridSize=(8, 8))
    enhanced = clahe.apply(smooth)

    # Thresholding
    blurred = cv2.GaussianBlur(enhanced, (3, 3), 0)
    _, otsu = cv2.threshold(blurred, 0, 255,
                            cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    adaptive = cv2.adaptiveThreshold(
        blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY_INV, 35, 2
    )

    thresh = cv2.bitwise_or(otsu, adaptive)

    # ---------------------------
    # NEW: Separate touching axons using WATERSHED
    # ---------------------------
    # thresh: foreground (axon structures) = 255, background = 0
    kernel = np.ones((3, 3), np.uint8)

    # Remove tiny noise, keep main blobs
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=1)

    # Sure background
    sure_bg = cv2.dilate(opening, kernel, iterations=3)

    # Sure foreground via distance transform
    dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 5)
    if dist_transform.max() > 0:
        _, sure_fg = cv2.threshold(dist_transform,
                                   0.4 * dist_transform.max(), 255, 0)
    else:
        sure_fg = np.zeros_like(opening)
    sure_fg = np.uint8(sure_fg)

    # Unknown region (possible boundaries between axons)
    unknown = cv2.subtract(sure_bg, sure_fg)

    # Marker labelling for watershed
    num_markers, markers = cv2.connectedComponents(sure_fg)
    markers = markers + 1  # background becomes 1 instead of 0
    markers[unknown == 255] = 0

    # Watershed needs 3-channel image
    wshed_input = cv2.cvtColor(enhanced, cv2.COLOR_GRAY2BGR)
    markers = cv2.watershed(wshed_input, markers)

    # Use watershed boundaries to carve gaps between touching axons
    cleaned = thresh.copy()
    cleaned[markers == -1] = 0  # make watershed boundary black (gap)
    # ---------------------------

    contours, hierarchy = cv2.findContours(
        cleaned, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE
    )
    if hierarchy is None:
        return object_counter
    hierarchy = hierarchy[0]

    for i, contour in enumerate(contours):

        # Only consider outermost contours
        if hierarchy[i][3] != -1:
            continue

        area = cv2.contourArea(contour)
        if area < MIN_CONTOUR_AREA or area > MAX_CONTOUR_AREA:
            continue

        overlap = contour_overlap_ratio(contour, mask_patch)
        if overlap < TISSUE_OVERLAP_MIN:
            continue

        hull = cv2.convexHull(contour)
        solidity = area / (cv2.contourArea(hull) + 1e-6)
        circ = circularity(contour)
        if solidity < MIN_SOLIDITY or circ < MIN_CIRCULARITY:
            continue

        # inner contours
        inner_children = []
        for j, child in enumerate(contours):
            if hierarchy[j][3] == i and cv2.contourArea(child) > 30:
                inner_children.append(child)

        if len(inner_children) == 0:
            continue

        axon_type = "mature" if len(inner_children) <= 2 else "regrowth_cluster"

        (x_outer, y_outer), outer_radius = cv2.minEnclosingCircle(contour)
        contour_shifted = contour + np.array([x_offset, y_offset])
        x_outer += x_offset
        y_outer += y_offset

        outer_area_px = cv2.contourArea(contour)
        outer_area_um2 = outer_area_px * (scale_factor ** 2)

        outer_color = (255, 0, 0) if axon_type == "mature" else (0, 255, 255)
        cv2.drawContours(output_image, [contour_shifted], -1, outer_color, 1)

        inner_radii = []
        inner_areas = []

        for inner_c in inner_children:
            (x_inner, y_inner), inner_radius = cv2.minEnclosingCircle(inner_c)
            inner_shifted = inner_c + np.array([x_offset, y_offset])
            cv2.drawContours(output_image, [inner_shifted], -1, (0, 255, 0), 1)
            inner_radii.append(inner_radius)
            inner_areas.append(cv2.contourArea(inner_c))

        largest_inner = inner_children[np.argmax(inner_areas)]
        inner_radius = max(inner_radii)
        inner_area_px = max(inner_areas)
        inner_area_um2 = inner_area_px * (scale_factor ** 2)

        thickness = (outer_radius - inner_radius) * scale_factor
        g_ratio = inner_radius / (outer_radius + 1e-6)
        area_ratio = inner_area_um2 / (outer_area_um2 + 1e-6)

        axon_data.append({
            "axon_id": object_counter,
            "axon_type": axon_type,
            "num_inner_contours": len(inner_children),
            "center_x": x_outer,
            "center_y": y_outer,
            "outer_radius": outer_radius * scale_factor,
            "inner_radius": inner_radius * scale_factor,
            "thickness": thickness,
            "diameter": (2 * outer_radius) * scale_factor,
            "outer_area": outer_area_um2,
            "inner_area": inner_area_um2,
            "area_ratio": area_ratio,
            "hole_ratio": inner_area_um2 / (outer_area_um2 + 1e-6),
            "eccentricity": 0,
            "g_ratio": g_ratio,
            "contour": contour_shifted.copy()
        })

        cv2.putText(output_image, str(object_counter),
                    (int(x_outer), int(y_outer)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

        object_counter += 1

    return object_counter



def compute_blur_mask_full_image(image, window_size=7, percentile=0):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray_denoised = cv2.GaussianBlur(gray, (3, 3), 0)

    lap = cv2.Laplacian(gray_denoised, cv2.CV_64F)
    lap = np.abs(lap)

    mean = cv2.blur(lap, (window_size, window_size))
    mean_sq = cv2.blur(lap ** 2, (window_size, window_size))
    var = mean_sq - mean ** 2
    var = cv2.normalize(var, None, 0, 255, cv2.NORM_MINMAX)
    blur_map = var.astype(np.uint8)

    thresh = np.percentile(blur_map, percentile)
    blur_mask = np.zeros_like(blur_map, dtype=np.uint8)
    blur_mask[blur_map <= thresh] = 255  # white = blurred

    return blur_map, blur_mask

# --------------------------
# Whole image processing
# --------------------------
def process_image(image_path, patch_size=1024):
    image = cv2.imread(image_path)
    if image is None:
        print(f"❌ Cannot read image: {image_path}")
        return None, None

    tissue_mask_full = build_tissue_mask(image)
    h, w = image.shape[:2]

    output_image = image.copy()
    axon_data = []
    object_counter = 1

    for y in range(0, h, patch_size):
        for x in range(0, w, patch_size):
            patch = image[y:y+patch_size, x:x+patch_size]
            mask_patch = tissue_mask_full[y:y+patch_size, x:x+patch_size]

            object_counter = process_patch(
                patch, mask_patch, x, y, axon_data, object_counter, output_image
            )

    print(f"Processed {len(axon_data)} axons.")
    blur_map, blur_mask = compute_blur_mask_full_image(image, 7, 0)

    filtered_axons = []
    removed_axons = []
    
    # for axon in axon_data:
    #     contour = axon["contour"]
    
    #     c_mask = np.zeros(blur_mask.shape, dtype=np.uint8)
    #     cv2.drawContours(c_mask, [contour], -1, 255, -1)
    
    #     overlap = cv2.bitwise_and(c_mask, blur_mask)
    #     overlap_area = cv2.countNonZero(overlap)
    
    #     contour_area = cv2.countNonZero(c_mask)
    
    #     # REMOVE only if axon is COMPLETELY inside blur
    #     # REMOVE only if 80% axon is inside blur
    #     if overlap_area >= 0.99 * contour_area:
    #         removed_axons.append(axon)
    #     else:
    #         filtered_axons.append(axon)

    for axon in axon_data:
        contour = axon["contour"]
    
        c_mask = np.zeros(blur_mask.shape, dtype=np.uint8)
        cv2.drawContours(c_mask, [contour], -1, 255, -1)
    
        overlap = cv2.bitwise_and(c_mask, blur_mask)
        overlap_area = cv2.countNonZero(overlap)
    
        if overlap_area == 0:
            filtered_axons.append(axon)
        else:
            removed_axons.append(axon)


    
    # ----------------------------------
    # Visualization images
    # ----------------------------------
    
    output_before = image.copy()
    output_after = image.copy()
    output_removed = image.copy()
    
    for axon in axon_data:
        cv2.drawContours(output_before, [axon["contour"]], -1, (0,255,0), 1)
    
    for axon in filtered_axons:
        cv2.drawContours(output_after, [axon["contour"]], -1, (0,255,0), 1)
    
    for axon in removed_axons:
        cv2.drawContours(output_removed, [axon["contour"]], -1, (0,0,255), 1)
    
    print(f"Axons before blur filtering: {len(axon_data)}")
    print(f"Axons removed due to blur: {len(removed_axons)}")
    print(f"Axons after blur filtering: {len(filtered_axons)}")
    
    return (
        axon_data,
        filtered_axons,
        output_before,
        output_after,
        output_removed,
        blur_map,
        blur_mask
    )

# --------------------------
# Save CSV
# --------------------------
def save_to_csv(axon_data, output_folder):
    df = pd.DataFrame(axon_data)
    csv_path = os.path.join(output_folder, "axon_measurements.csv")
    df.to_csv(csv_path, index=False)
    return df

# --------------------------
# Visualizations
# --------------------------
def plot_and_visualize(df, output_folder):
    numeric_cols = df.select_dtypes(include=[np.number]).columns

    for col in numeric_cols:
        vals = df[col].dropna()
        if len(vals) == 0:
            continue

        plt.hist(vals, bins=25)
        plt.title(col)
        plt.savefig(os.path.join(output_folder, f"hist_{col}.png"))
        plt.close()

# --------------------------
# Process entire folder
# --------------------------
def process_folder(input_folder, output_root):
    images = [
        f for f in os.listdir(input_folder)
        if f.lower().endswith((".png", ".jpg", ".jpeg", ".bmp"))
    ]

    for img_name in images:
        print("\n==============================")
        print(f"Processing {img_name}")
        print("==============================")

        img_path = os.path.join(input_folder, img_name)
        img_name_wo_ext = os.path.splitext(img_name)[0]
        img_output_folder = os.path.join(output_root, img_name_wo_ext)
        os.makedirs(img_output_folder, exist_ok=True)

        result = process_image(img_path)

        if result is None:
            continue

        (axon_data,
         filtered_axons,
         output_before,
         output_after,
         output_removed,
         blur_map,
         blur_mask) = result

        # -------------------------------------------------
        # SAVE CSV (Filtered axons only – recommended)
        # -------------------------------------------------
        df = save_to_csv(filtered_axons, img_output_folder)
        plot_and_visualize(df, img_output_folder)

        # -------------------------------------------------
        # SAVE INTERMEDIARY IMAGES
        # -------------------------------------------------
        cv2.imwrite(os.path.join(img_output_folder, "01_before_blur_filter.png"), output_before)
        cv2.imwrite(os.path.join(img_output_folder, "02_removed_due_to_blur.png"), output_removed)
        cv2.imwrite(os.path.join(img_output_folder, "03_after_blur_filter.png"), output_after)
        cv2.imwrite(os.path.join(img_output_folder, "04_blur_map.png"), blur_map)
        cv2.imwrite(os.path.join(img_output_folder, "05_blur_mask.png"), blur_mask)

        # -------------------------------------------------
        # CREATE SUMMARY FIGURE
        # -------------------------------------------------
        original = cv2.imread(img_path)

        plt.figure(figsize=(18, 10))

        plt.subplot(2, 3, 1)
        plt.imshow(cv2.cvtColor(original, cv2.COLOR_BGR2RGB))
        plt.title("Original")
        plt.axis("off")

        plt.subplot(2, 3, 2)
        plt.imshow(blur_map, cmap='gray')
        plt.title("Blur Map")
        plt.axis("off")

        plt.subplot(2, 3, 3)
        plt.imshow(blur_mask, cmap='gray')
        plt.title("Blur Mask (White = Blurred)")
        plt.axis("off")

        plt.subplot(2, 3, 4)
        plt.imshow(cv2.cvtColor(output_before, cv2.COLOR_BGR2RGB))
        plt.title("Detected Axons (Before)")
        plt.axis("off")

        plt.subplot(2, 3, 5)
        plt.imshow(cv2.cvtColor(output_removed, cv2.COLOR_BGR2RGB))
        plt.title("Removed Due to Blur")
        plt.axis("off")

        plt.subplot(2, 3, 6)
        plt.imshow(cv2.cvtColor(output_after, cv2.COLOR_BGR2RGB))
        plt.title("Final Axons (After)")
        plt.axis("off")

        plt.tight_layout()
        plt.savefig(os.path.join(img_output_folder, "06_intermediary_summary.png"))
        plt.close()

        print(f"✔ Saved results for {img_name} in {img_output_folder}\n")

process_folder(input_folder, output_root)
print("\n🎉 DONE! All images processed.")





Processing 1.png
Processed 192 axons.
Axons before blur filtering: 192
Axons removed due to blur: 156
Axons after blur filtering: 36
✔ Saved results for 1.png in C:\Users\3kp05\Downloads\removed if touches blur\1


Processing 2.png
Processed 340 axons.
Axons before blur filtering: 340
Axons removed due to blur: 230
Axons after blur filtering: 110
✔ Saved results for 2.png in C:\Users\3kp05\Downloads\removed if touches blur\2


Processing 3.png
Processed 535 axons.
Axons before blur filtering: 535
Axons removed due to blur: 318
Axons after blur filtering: 217
✔ Saved results for 3.png in C:\Users\3kp05\Downloads\removed if touches blur\3


Processing 4.png
Processed 432 axons.
Axons before blur filtering: 432
Axons removed due to blur: 222
Axons after blur filtering: 210
✔ Saved results for 4.png in C:\Users\3kp05\Downloads\removed if touches blur\4


Processing 5.png
Processed 277 axons.
Axons before blur filtering: 277
Axons removed due to blur: 135
Axons after blur filtering: 142
✔ 